In [1]:
import pandas as pd
import imblearn

In [2]:
data = pd.read_excel("Raw Data.xlsx")
data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43395,56196,Female,10.0,0,0,No,children,Urban,58.64,20.4,never smoked,0
43396,5450,Female,56.0,0,0,Yes,Govt_job,Urban,213.61,55.4,formerly smoked,0
43397,28375,Female,82.0,1,0,Yes,Private,Urban,91.94,28.9,formerly smoked,0
43398,27973,Male,40.0,0,0,Yes,Private,Urban,99.16,33.2,never smoked,0


In [ ]:
data.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,43400.000000,43400.000000,43400.000000,43400.000000,43400.000000,41938.000000,43400.000000
mean,36326.142350,42.217894,0.093571,0.047512,104.482750,28.605038,0.018041
std,21072.134879,22.519649,0.291235,0.212733,43.111751,7.770020,0.133103
min,1.000000,0.080000,0.000000,0.000000,55.000000,10.100000,0.000000
25%,18038.500000,24.000000,0.000000,0.000000,77.540000,23.200000,0.000000
50%,36351.500000,44.000000,0.000000,0.000000,91.580000,27.700000,0.000000
75%,54514.250000,60.000000,0.000000,0.000000,112.070000,32.900000,0.000000
max,72943.000000,82.000000,1.000000,1.000000,291.050000,97.600000,1.000000


In [ ]:
data['smoking_status'].unique()

array([nan, 'never smoked', 'formerly smoked', 'smokes'], dtype=object)

In [3]:
import numpy as np

def categorize_blood_pressure(sys_value, dia_value):
    conditions = [
        (sys_value < 120) & (dia_value < 80),
        (sys_value >= 120) & (sys_value < 130) & (dia_value < 80),
        ((sys_value >= 130) & (sys_value < 140)) | ((dia_value >= 80) & (dia_value < 90)),
        (sys_value >= 140) | (dia_value >= 90),
        (sys_value > 180) | (dia_value > 120)
    ]

    categories = [
        'Normal',
        'Elevated',
        'High Blood Pressure (Hypertension) Stage 1',
        'High Blood Pressure (Hypertension) Stage 2',
        'Hypertensive Crisis'
    ]

    # Initialize hypertension flag
    hypertension_flag = 0

    # Set hypertension flag if blood pressure falls into stage 1 or 2 categories
    if conditions[2] or conditions[3]:
        hypertension_flag = 1

    # Categorize blood pressure
    bp_category = np.select(conditions, categories, default='Unknown')

    return hypertension_flag


In [ ]:
categorize_blood_pressure(110,70)

0

In [4]:
data.isnull().sum()

id                       0
gender                   0
age                      0
hypertension             0
heart_disease            0
ever_married             0
work_type                0
Residence_type           0
avg_glucose_level        0
bmi                   1462
smoking_status       13292
stroke                   0
dtype: int64

In [4]:
data['age'] = data['age'].astype(int)

In [5]:
df = data[data['age']>=18]

In [6]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
1,30468,Male,58,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
3,56543,Female,70,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
5,32257,Female,47,0,0,Yes,Private,Urban,210.95,50.1,NaN,0
6,52800,Female,52,0,0,Yes,Private,Urban,77.59,17.7,formerly smoked,0
7,41413,Female,75,0,1,Yes,Self-employed,Rural,243.53,27.0,never smoked,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43394,52380,Male,47,0,0,No,Govt_job,Urban,68.52,25.2,formerly smoked,0
43396,5450,Female,56,0,0,Yes,Govt_job,Urban,213.61,55.4,formerly smoked,0
43397,28375,Female,82,1,0,Yes,Private,Urban,91.94,28.9,formerly smoked,0
43398,27973,Male,40,0,0,Yes,Private,Urban,99.16,33.2,never smoked,0


In [6]:
def classify_glucose_level(glucose_level):
    if 80 <= glucose_level <= 100:
        return 1  # Normal
    elif 101 <= glucose_level <= 125:
        return 2  # Above Normal
    elif glucose_level >= 126:
        return 3  # Well Above Normal
    else:
        return 0  # Value below normal range

In [7]:
df['glucose_level'] = df['avg_glucose_level'].apply(classify_glucose_level)

<ipython-input-7-1bc6c64962b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['glucose_level'] = df['avg_glucose_level'].apply(classify_glucose_level)


In [9]:
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,glucose_level
1,30468,Male,58,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0,1
3,56543,Female,70,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0,0
5,32257,Female,47,0,0,Yes,Private,Urban,210.95,50.1,NaN,0,3
6,52800,Female,52,0,0,Yes,Private,Urban,77.59,17.7,formerly smoked,0,0
7,41413,Female,75,0,1,Yes,Self-employed,Rural,243.53,27.0,never smoked,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43394,52380,Male,47,0,0,No,Govt_job,Urban,68.52,25.2,formerly smoked,0,0
43396,5450,Female,56,0,0,Yes,Govt_job,Urban,213.61,55.4,formerly smoked,0,3
43397,28375,Female,82,1,0,Yes,Private,Urban,91.94,28.9,formerly smoked,0,1
43398,27973,Male,40,0,0,Yes,Private,Urban,99.16,33.2,never smoked,0,1


In [ ]:
df.isnull().sum()

id                      0
gender                  0
age                     0
hypertension            0
heart_disease           0
ever_married            0
work_type               0
Residence_type          0
avg_glucose_level       0
bmi                  1350
smoking_status       7174
stroke                  0
glucose_level           0
dtype: int64

In [8]:
df['bmi'].fillna(df['bmi'].median(),inplace=True)

<ipython-input-8-efbe5f93a4a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bmi'].fillna(df['bmi'].median(),inplace=True)


In [ ]:
df['bmi'].fillna(df['bmi'].median(),inplace=True)

<ipython-input-333-efbe5f93a4a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bmi'].fillna(df['bmi'].median(),inplace=True)


In [9]:
df['smoking_status'].fillna(df['smoking_status'].mode()[0],inplace=True)

<ipython-input-9-6054fc301164>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['smoking_status'].fillna(df['smoking_status'].mode()[0],inplace=True)


In [11]:
df.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
glucose_level        0
dtype: int64

In [10]:
def categorize_age(df, age_column, bins, labels):
  if age_column not in df.columns:
        raise ValueError(f"The column '{age_column}' does not exist in the DataFrame.")

    # Use pd.cut to categorize ages
  df['age_category'] = pd.cut(df[age_column], bins=bins, labels=labels,right=False, include_lowest=True)

  return df

In [11]:
bins = [18, 30, 40, 50,60,70,80,101]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69','70-79','80-100']
df = categorize_age(df, 'age', bins, labels)

<ipython-input-10-d3de2844b11a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age_category'] = pd.cut(df[age_column], bins=bins, labels=labels,right=False, include_lowest=True)


In [12]:
def categorize_bmi(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 25:
        return "Normal weight"
    elif 25 <=bmi< 30:
        return "Overweight"
    elif bmi>=30:
      return "Obese"

In [13]:
# Assuming 'df' is your DataFrame and 'bmi_column' is the column containing BMI values
df['BMI_Category'] = df['bmi'].apply(categorize_bmi)

<ipython-input-13-683fa09aa204>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BMI_Category'] = df['bmi'].apply(categorize_bmi)


In [16]:
df['BMI_Category'].unique()

array(['Obese', 'Underweight', 'Overweight', 'Normal weight'],
      dtype=object)

In [14]:
df.drop(columns=['bmi','age','id'],axis=1,inplace = True)

<ipython-input-14-743fe12b82ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['bmi','age','id'],axis=1,inplace = True)


In [15]:
df.drop(['work_type'],axis=1,inplace=True)

<ipython-input-15-67ecbc088232>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['work_type'],axis=1,inplace=True)


In [16]:
df.drop(['Residence_type'],axis=1,inplace=True)

<ipython-input-16-f265b6932d01>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Residence_type'],axis=1,inplace=True)


In [17]:
df.drop(['ever_married'],axis=1,inplace=True)

<ipython-input-17-5b2158b706ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['ever_married'],axis=1,inplace=True)


In [18]:
df.drop(['avg_glucose_level'],axis=1,inplace=True)

<ipython-input-18-b0dc84379f4f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['avg_glucose_level'],axis=1,inplace=True)


In [ ]:
df['gender'].unique()

array(['Male', 'Female', 'Other'], dtype=object)

In [19]:
df = df[df['gender'] != 'Other']

In [23]:
df

,gender,hypertension,heart_disease,smoking_status,stroke,glucose_level,age_category,BMI_Category
1,Male,1,0,never smoked,0,1,50-59,Obese
3,Female,0,0,formerly smoked,0,0,70-79,Obese
5,Female,0,0,never smoked,0,3,40-49,Obese
6,Female,0,0,formerly smoked,0,0,50-59,Underweight
7,Female,0,1,never smoked,0,3,70-79,Overweight
...,...,...,...,...,...,...,...,...
43394,Male,0,0,formerly smoked,0,0,40-49,Overweight
43396,Female,0,0,formerly smoked,0,3,50-59,Obese
43397,Female,1,0,formerly smoked,0,1,80-100,Overweight
43398,Male,0,0,never smoked,0,1,40-49,Obese


In [20]:
from sklearn.preprocessing import OneHotEncoder
categorical_columns = ['BMI_Category','age_category','gender','smoking_status']
encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(df[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

df.reset_index(drop=True, inplace=True)
one_hot_df.reset_index(drop=True, inplace=True)

data_encoded = pd.concat([df, one_hot_df], axis=1)
data_encoded = data_encoded.drop(categorical_columns, axis=1)

data_encoded

,hypertension,heart_disease,stroke,glucose_level,BMI_Category_Normal weight,BMI_Category_Obese,BMI_Category_Overweight,BMI_Category_Underweight,age_category_18-29,age_category_30-39,age_category_40-49,age_category_50-59,age_category_60-69,age_category_70-79,age_category_80-100,gender_Female,gender_Male,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,1,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0,0,0,3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0,1,0,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35845,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
35846,0,0,0,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
35847,1,0,0,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
35848,0,0,0,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [ ]:
df['heart_disease'].unique()

array([0, 1])

In [ ]:
'''from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
column_data = data_encoded['avg_glucose_level'].values.reshape(-1, 1)
data_encoded['avg_glucose_level'] = scaler.fit_transform(column_data)'''

"from sklearn.preprocessing import MinMaxScaler\nscaler = MinMaxScaler()\ncolumn_data = data_encoded['avg_glucose_level'].values.reshape(-1, 1)\ndata_encoded['avg_glucose_level'] = scaler.fit_transform(column_data)"

In [25]:
data_encoded[data_encoded['stroke']==1]

,hypertension,heart_disease,stroke,glucose_level,BMI_Category_Normal weight,BMI_Category_Obese,BMI_Category_Overweight,BMI_Category_Underweight,age_category_18-29,age_category_30-39,age_category_40-49,age_category_50-59,age_category_60-69,age_category_70-79,age_category_80-100,gender_Female,gender_Male,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
53,0,1,1,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
69,0,0,1,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
121,0,1,1,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
217,0,0,1,3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
224,1,0,1,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35649,0,0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
35747,0,0,1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
35775,0,0,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
35787,1,1,1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [21]:
from sklearn.model_selection import train_test_split

X = data_encoded.drop('stroke', axis=1)
y = data_encoded['stroke']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [22]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(0, 35069), (1, 35069)]


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.33, random_state=42)

In [24]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score

classifiers = [
    KNeighborsClassifier(2),
    SVC(gamma=2, C=1, random_state=42),
    DecisionTreeClassifier(max_depth=6, random_state=42),
    RandomForestClassifier(
        max_depth=5, n_estimators=6, max_features=1, random_state=42
    ),
    AdaBoostClassifier(algorithm="SAMME", random_state=42),
]

In [30]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize the Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=300, learning_rate=0.35, max_depth=11)

# Train the classifier
gb_clf.fit(X_train, y_train)

# Predict on the test set
y_pred = gb_clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8258446383824419


In [47]:
import xgboost as xgb

# Initialize the XGBoost Classifier
xgb_clf = xgb.XGBClassifier(n_estimators=250, learning_rate=0.47, max_depth=9)

# Train the classifier
xgb_clf.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8253693942797892


In [26]:
for clf in classifiers:
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  cm = confusion_matrix(y_test, y_pred)
  acc = accuracy_score(y_test,y_pred)
  rec = recall_score(y_test,y_pred,average='micro')
  p = precision_score(y_test,y_pred, average='micro')
  print("Accuracy: ",acc)
  print("Recall: ",rec)
  print("-------------------------")

Accuracy:  0.730363777758576
Recall:  0.730363777758576
-------------------------
Accuracy:  0.825931046401106
Recall:  0.825931046401106
-------------------------
Accuracy:  0.7607794003283505
Recall:  0.7607794003283505
-------------------------
Accuracy:  0.7274259051239955
Recall:  0.7274259051239955
-------------------------
Accuracy:  0.7559405512831591
Recall:  0.7559405512831591
-------------------------


In [25]:
model = SVC(gamma=3, C=5, random_state=42,kernel='rbf')
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test,y_pred)
rec = recall_score(y_test,y_pred,average='micro')
print(f"Recall: {rec}\nAccuracy: {acc}")

Recall: 0.825931046401106
Accuracy: 0.825931046401106


In [34]:
df

,gender,hypertension,heart_disease,smoking_status,stroke,glucose_level,age_category,BMI_Category
0,Male,1,0,never smoked,0,1,50-59,Obese
1,Female,0,0,formerly smoked,0,0,70-79,Obese
2,Female,0,0,never smoked,0,3,40-49,Obese
3,Female,0,0,formerly smoked,0,0,50-59,Underweight
4,Female,0,1,never smoked,0,3,70-79,Overweight
...,...,...,...,...,...,...,...,...
35845,Male,0,0,formerly smoked,0,0,40-49,Overweight
35846,Female,0,0,formerly smoked,0,3,50-59,Obese
35847,Female,1,0,formerly smoked,0,1,80-100,Overweight
35848,Male,0,0,never smoked,0,1,40-49,Obese


In [26]:
data = {
    'hypertension': [1],
    'heart_disease': [0],
    'glucose_level': [3],
    'smoking_status':['never smoked'],
    'BMI_Category':['Normal weight'],
    'gender':['Female'],
    'age_category':['70-79']
}

dss = pd.DataFrame(data)

In [27]:

categorical_columns = ['BMI_Category','age_category','gender','smoking_status']
one_hot_encoded = encoder.transform(dss[categorical_columns].astype('category'))
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

df_encoded = pd.concat([dss, one_hot_df], axis=1)
df_encoded = df_encoded.drop(categorical_columns, axis=1)

In [ ]:
df_encoded

,hypertension,heart_disease,glucose_level,BMI_Category_Normal weight,BMI_Category_Obese,BMI_Category_Overweight,BMI_Category_Underweight,age_category_18-29,age_category_30-39,age_category_40-49,age_category_50-59,age_category_60-69,age_category_70-79,age_category_80-100,gender_Female,gender_Male,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,1,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [32]:
gb_clf.predict(df_encoded)

array([1])

In [48]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Select top k features based on mutual information
k = 10  # Number of features to select
selector = SelectKBest(mutual_info_classif, k=k)
selector.fit(X_train, y_train)

# Transform the dataset to include only the selected features
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)
selected_feature_indices = selector.get_support(indices=True)
selected_features = X_train.columns[selected_feature_indices]  # Assuming X_train is a DataFrame
print("Selected Features:")
print(selected_features)


Selected Features:
Index(['hypertension', 'heart_disease', 'glucose_level',
       'BMI_Category_Overweight', 'age_category_18-29', 'age_category_30-39',
       'age_category_40-49', 'age_category_70-79', 'age_category_80-100',
       'smoking_status_never smoked'],
      dtype='object')


In [ ]:
import joblib

# Assuming 'model' is your trained scikit-learn model
joblib.dump(model, 'brainstrokemodel.pkl')

['brainstrokemodel.pkl']

In [ ]:
joblib.dump(encoder,'brainstrokeEncoder.pkl')

['brainstrokeEncoder.pkl']

In [ ]:
joblib.dump(scaler,'brainstrokeScaler.pkl')

['brainstrokeScaler.pkl']